<a href="https://colab.research.google.com/github/Anubhab2002/timeVAE-gen-eval/blob/main/timeVAE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/abudesai/timeVAE.git
%cd timeVAE

Cloning into 'timeVAE'...
remote: Enumerating objects: 89, done.
remote: Counting objects: 100% (89/89), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 89 (delta 27), reused 66 (delta 12), pack-reused 0
Unpacking objects: 100% (89/89), done.
/content/timeVAE/timeVAE


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
%matplotlib notebook

# Data Subsample Generation - 15%


In [ ]:
# create data for 15% subsampled sine data

import numpy as np, pandas as pd
import os, sys
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import joblib
from datetime import datetime, date, time, timedelta
import utils

In [ ]:
output_dir = "/content/timeVAE/datasets"

In [ ]:
selected = 'sine'


In [ ]:
def gen_sine_data(no, seq_len, dim):   
    """Sine data generation.
    Args:
    - no: the number of samples
    - seq_len: sequence length of the time-series
    - dim: feature dimensions

    Returns:
    - data: generated data
    """ 
    size = (no, 1, dim)
    freq = np.random.uniform(0.1, 0.15, 1)  
    
    
    phase = np.random.uniform(0., 2 * np.pi, size) 
    amplitude = np.random.uniform(1, 3, size) 
    
    seq = np.arange(seq_len)
    seq = np.expand_dims(seq, axis=0)
    seq = np.expand_dims(seq, axis=-1)
    
    data = amplitude * np.sin(2 * np.pi * freq * seq + phase)    
    data = (data + 1) * 0.5
    return data

# testing for correlations
np.random.seed(0)
N, T, D = 10000, 24, 5 
sine_data = gen_sine_data(N, 24, D)
sine_data = np.array(sine_data)
sine_data = sine_data[0].reshape((-1, D))
np.corrcoef(sine_data.T)

array([[ 1.        ,  0.7607258 ,  0.4744157 , -0.27526657,  0.90768667],
       [ 0.7607258 ,  1.        ,  0.93228017,  0.41459596,  0.96288337],
       [ 0.4744157 ,  0.93228017,  1.        ,  0.71570232,  0.80003782],
       [-0.27526657,  0.41459596,  0.71570232,  1.        ,  0.15358086],
       [ 0.90768667,  0.96288337,  0.80003782,  0.15358086,  1.        ]])

In [ ]:
np.random.seed(42)  # for reproducibility

N, T, D = 10000, 24, 5 
TEST_PERC = 0.00
TRAIN_PERC = 1- TEST_PERC

perc_of_train_data = [2, 5, 10, 15, 20, 100]

full_sine_data = gen_sine_data(N, T, D)
np.random.shuffle(full_sine_data)    

for p in perc_of_train_data:
    N_small = int(N * p / 100)
    sub_sine_data = full_sine_data[np.random.choice(N, N_small, replace=False)]   
    print(sub_sine_data.shape)
    
    fname = f'{selected}_subsampled_train_perc_{p}.npz'
    full_path = os.path.join(output_dir, fname)
    print(full_path)
    
#     np.save(full_path, sine_data)
    np.savez_compressed(full_path, data=sub_sine_data)

(200, 24, 5)
/content/timeVAE/datasets/sine_subsampled_train_perc_2.npz
(500, 24, 5)
/content/timeVAE/datasets/sine_subsampled_train_perc_5.npz
(1000, 24, 5)
/content/timeVAE/datasets/sine_subsampled_train_perc_10.npz
(1500, 24, 5)
/content/timeVAE/datasets/sine_subsampled_train_perc_15.npz
(2000, 24, 5)
/content/timeVAE/datasets/sine_subsampled_train_perc_20.npz
(10000, 24, 5)
/content/timeVAE/datasets/sine_subsampled_train_perc_100.npz


In [ ]:
np.random.seed(0)
selected = 'sine'
perc = 10

loaded = np.load(f'{output_dir}/{selected}_subsampled_train_perc_{perc}.npz')
data = loaded['data']
# print(data.shape)

# print("data mean : ", data.mean(axis=0).mean(axis=0)) 

utils.plot_samples(data, n=5)

<IPython.core.display.Javascript object>

# Synthetic Data Generation

In [ ]:
!mkdir outputs
!mkdir model

In [75]:
!python test_vae.py

data shape: 1500 24 5
train/valid shapes:  (1350, 24, 5) (150, 24, 5)
Epoch 1/500
43/43 [==============================] - 2s 13ms/step - loss: 6966.7545 - reconstruction_loss: 1003.3138 - kl_loss: 299.2665 - lr: 0.0010
Epoch 2/500
43/43 [==============================] - 0s 11ms/step - loss: 990.5866 - reconstruction_loss: 269.2555 - kl_loss: 142.2401 - lr: 0.0010
Epoch 3/500
43/43 [==============================] - 1s 14ms/step - loss: 929.0971 - reconstruction_loss: 260.7231 - kl_loss: 129.4245 - lr: 0.0010
Epoch 4/500
43/43 [==============================] - 1s 13ms/step - loss: 911.7604 - reconstruction_loss: 258.1153 - kl_loss: 119.9280 - lr: 0.0010
Epoch 5/500
43/43 [==============================] - 1s 12ms/step - loss: 882.9622 - reconstruction_loss: 253.8790 - kl_loss: 113.6231 - lr: 0.0010
Epoch 6/500
43/43 [==============================] - 1s 12ms/step - loss: 886.1930 - reconstruction_loss: 252.2980 - kl_loss: 102.1636 - lr: 0.0010
Epoch 7/500
43/43 [=====================

# MSE Loss Calculation

In [28]:
import tensorflow as tf
import utils

In [29]:
def get_data(input_file):
    loaded = np.load(input_file)
    return loaded['data']

In [76]:
input_file = '/content/timeVAE/datasets/sine_subsampled_train_perc_15.npz'
output_file = "/content/timeVAE/outputs/timeVAE_gen_samples_sine_perc_15.npz"

original_data = get_data(input_file)
synthetic_data = get_data(output_file)

print(original_data.shape, synthetic_data.shape)




(1500, 24, 5) (1500, 24, 5)


In [78]:
mse_loss=np.mean(np.power((original_data-synthetic_data),2))

In [79]:
mse_loss

0.10702251601824057

In [68]:
print("MSE LOSS AT 5%:", mse_loss)

MSE LOSS AT 5%: 0.27210297651397797


In [74]:
print("MSE LOSS AT 10%:", mse_loss)

MSE LOSS AT 10%: 0.10209742608147447


In [80]:
print("MSE LOSS AT 15%:", mse_loss)

MSE LOSS AT 15%: 0.10702251601824057


In [57]:
print("MSE LOSS AT 20%:", mse_loss)

MSE LOSS AT 20%: 0.1062706584229444


In [63]:
class MinMaxScaler():
    """Min Max normalizer.
    Args:
    - data: original data

    Returns:
    - norm_data: normalized data
    """
    def fit_transform(self, data): 
        self.fit(data)
        scaled_data = self.transform(data)
        return scaled_data


    def fit(self, data):    
        self.mini = np.min(data, 0)
        self.range = np.max(data, 0) - self.mini
        return self
        

    def transform(self, data):
        numerator = data - self.mini
        scaled_data = numerator / (self.range + 1e-7)
        return scaled_data

    
    def inverse_transform(self, data):
        data *= self.range
        data += self.mini
        return data

In [64]:
def scale_data(orig, gen, scale_gen_data=True): 
    scaler_orig = MinMaxScaler( )  
    scaled_orig_data = scaler_orig.fit_transform(orig)
    if scale_gen_data: 
        scaled_gen_data = scaler_orig.transform(gen)
    else: scaled_gen_data = gen
    return scaled_orig_data, scaled_gen_data

In [77]:
original_data, synthetic_data = scale_data(original_data, synthetic_data)

# t-SNE Plots for synthetic vs original data

In [84]:
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import numpy as np, pandas as pd
import utils
import sys, os

In [81]:
def avg_over_feat_dim(orig, gen): 
    final_orig = orig.mean(axis=2)
    final_gen = gen.mean(axis=2)
    return final_orig, final_gen

In [82]:
def shuffle_and_subsample(orig, gen, num=1000):     
    np.random.shuffle(orig)
    np.random.shuffle(gen)
    final_orig = orig[:num]
    final_gen = gen[:num]
    return final_orig, final_gen

In [83]:
def print_all_data():
    print(orig_data.mean(axis=0).mean(axis=1))
    print(gen_data.mean(axis=0).mean(axis=1))
    
    print('scaled orig and gen data shape: ', scaled_orig_data.shape, scaled_gen_data.shape)
    print('scaled orig data mean: ', scaled_orig_data.mean())
    print('scaled gen data mean: ', scaled_gen_data.mean())
    print('final_orig shape: ', final_orig.shape)
    print('final_gen shape: ', final_gen.shape)

In [94]:
percs = [5,10,15,20]
datasets = ['sine']

figsize = ( len(datasets) * 2, 1 + len(percs) * 2)   # width by height
fig, axs = plt.subplots(len(percs), len(datasets), figsize=figsize)

for i, dataset in enumerate(datasets): 
  for j, perc in enumerate(percs):
    orig_data = get_data(f'/content/timeVAE/datasets/sine_subsampled_train_perc_{perc}.npz')
    gen_data = get_data(f'/content/timeVAE/outputs/timeVAE_gen_samples_sine_perc_{perc}.npz')
  #         print("orig, gen shapes: ", dataset, perc, orig_data.shape, gen_data.shape)

    if gen_data is None: continue
    
    N, T, D = orig_data.shape
    scaled_orig_data, scaled_gen_data = scale_data(orig_data, gen_data, scale_gen_data = True) 
    
    
    scaled_orig_2d, scaled_gen_2d = avg_over_feat_dim(scaled_orig_data, scaled_gen_data) 
    final_orig, final_gen = shuffle_and_subsample(scaled_orig_2d, scaled_gen_2d)        
    
    if final_orig.shape[0] != final_gen.shape[0]:
        final_orig = final_orig[:final_gen.shape[0]]
    
    N, T = final_orig.shape
    final_tsne_input = np.concatenate((final_orig, final_gen), axis = 0)        
  #         print('final_tsne_input shape: ', final_tsne_input.shape)

    # TSNE analysis
    tsne = TSNE(n_components = 2, perplexity = 40, n_iter = 300)
    tsne_results = tsne.fit_transform(final_tsne_input)

    colors = ["red" for i in range(N)] + ["blue" for i in range(N)] 

    axs[j].scatter(tsne_results[:N,0], tsne_results[:N,1], 
                c = colors[:N], alpha = 0.2)
    axs[j].scatter(tsne_results[N:,0], tsne_results[N:,1], 
                c = colors[N:], alpha = 0.2)
    
flat_labels = [(xl, yl) for xl in percs for yl in datasets]

for i, ax in enumerate(axs.flat):
    perc, dataset = flat_labels[i]
    ax.set_title(f'{dataset} | p = {perc}')

# fig.legend()
model = "timeVAE"
fig.tight_layout()
plt.subplots_adjust(top=0.93)
plt.suptitle(f'{model} t-SNE plot')
plt.savefig(f"{model}_tsne_results.png", dpi=300)  
plt.show()


<IPython.core.display.Javascript object>

In [96]:
!zip -r /content/timeVAE.zip /content/timeVAE

  adding: content/timeVAE/ (stored 0%)
  adding: content/timeVAE/trained_models/ (stored 0%)
  adding: content/timeVAE/trained_models/__init__.py (stored 0%)
  adding: content/timeVAE/vae_dense_model.py (deflated 67%)
  adding: content/timeVAE/loss_functions.py (deflated 59%)
  adding: content/timeVAE/test_vae.py (deflated 66%)
  adding: content/timeVAE/.ipynb_checkpoints/ (stored 0%)
  adding: content/timeVAE/.ipynb_checkpoints/time_gan-checkpoint.ipynb (deflated 42%)
  adding: content/timeVAE/outputs/ (stored 0%)
  adding: content/timeVAE/outputs/timeVAE_gen_samples_sine_perc_5.npz (deflated 0%)
  adding: content/timeVAE/outputs/timeVAE_gen_samples_sine_perc_15.npz (deflated 0%)
  adding: content/timeVAE/outputs/timeVAE_gen_samples_sine_perc_20.npz (deflated 0%)
  adding: content/timeVAE/outputs/timeVAE_gen_samples_sine_perc_10.npz (deflated 0%)
  adding: content/timeVAE/README.md (deflated 53%)
  adding: content/timeVAE/utils.py (deflated 68%)
  adding: content/timeVAE/.gitignore (s